In [ ]:
# Cell 1: Setup and Dependencies

# Install necessary libraries
!pip install pandas numpy torch torch_geometric tqdm

# --- Create required directories ---
# The scripts assume these folders exist for input files.
import os
os.makedirs("vocab", exist_ok=True)
os.makedirs("SnomedCT/Snapshot/Terminology", exist_ok=True)

print("Environment setup complete. Folders created.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.3 MB/s eta 0:00:00
Environment setup complete. Folders created.


In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# --- User Configuration: Set your base path for data on Google Drive ---
# Make sure your files are organized relative to this path, matching the original structure.
# Example: If 'mimic-iv-3.1' is in 'MyDrive/KnowledgeGraphData/', set path below.
BASE_DRIVE_PATH = '/content/drive/MyDrive/knowledgegraphdata'

# --- Placeholder creation removed ---
# You should place your actual data files in your Google Drive
# under the structure:
# {BASE_DRIVE_PATH}/mimic-iv-3.1/hosp/diagnoses_icd.csv.gz
# {BASE_DRIVE_PATH}/vocab/CONCEPT.zip
# {BASE_DRIVE_PATH}/vocab/CONCEPT_RELATIONSHIP.zip
# {BASE_DRIVE_PATH}/SnomedCT/Snapshot/Terminology/sct2_Relationship_Snapshot_US1000124_20250901.txt
# {BASE_DRIVE_PATH}/SnomedCT/Snapshot/Terminology/sct2_Description_Snapshot-en_US1000124_20250901.txt

print(f"Google Drive mounted. Please ensure your data files are in '{BASE_DRIVE_PATH}' and update relevant paths in subsequent cells.")

Mounted at /content/drive
Google Drive mounted. Please ensure your data files are in '/content/drive/MyDrive/knowledgegraphdata' and update relevant paths in subsequent cells.


In [ ]:
import pandas as pd

# === File paths ===
# Use BASE_DRIVE_PATH for Google Drive files
rels_path = os.path.join(BASE_DRIVE_PATH, "/content/drive/MyDrive/knowledgegraphdata/sct2_Relationship_Snapshot_US1000124_20250901.txt")
desc_path = os.path.join(BASE_DRIVE_PATH, "/content/drive/MyDrive/knowledgegraphdata/sct2_Description_Snapshot-en_US1000124_20250901.txt")

print("\n Loading SNOMED CT raw files...")
snomed_rels = pd.read_csv(rels_path, sep="\t", dtype=str)
snomed_descs = pd.read_csv(desc_path, sep="\t", dtype=str)
print(f"\u2192 Relationships: {snomed_rels.shape}, Descriptions: {snomed_descs.shape}")

# ===  Keep only active relationships ===
before_active = len(snomed_rels)
snomed_rels = snomed_rels[snomed_rels["active"] == "1"]
after_active = len(snomed_rels)
print(f" Active filter: {before_active:,} \u2192 {after_active:,} (removed {before_active - after_active:,})")

# ===  Extract preferred terms ===
before_pref = len(snomed_descs)
pref_terms = snomed_descs[snomed_descs["typeId"] == "900000000000013009"][["conceptId", "term"]]
pref_terms.dropna(subset=["term"], inplace=True)
after_pref = len(pref_terms)
print(f" Preferred terms: {before_pref:,} \u2192 {after_pref:,} (removed {before_pref - after_pref:,})")

# ===  Build full relation map (typeId \u2192 term) ===
relation_ids = snomed_rels["typeId"].unique()
relation_map_df = pref_terms[pref_terms["conceptId"].isin(relation_ids)]
relation_map = dict(zip(relation_map_df["conceptId"], relation_map_df["term"]))
print(f" Found {len(relation_map)} unique SNOMED relation types")

# === Add human-readable relation names ===
snomed_rels["relation_term"] = snomed_rels["typeId"].map(relation_map).fillna("other_relation")

# ===  Merge source & destination names ===
print("\n Merging concept names...")
rels_before_merge = snomed_rels.shape

snomed_rels = snomed_rels.merge(
    pref_terms, left_on="sourceId", right_on="conceptId", how="left"
).rename(columns={"term": "source_term"}).drop(columns=["conceptId"])

snomed_rels = snomed_rels.merge(
    pref_terms, left_on="destinationId", right_on="conceptId", how="left"
).rename(columns={"term": "destination_term"}).drop(columns=["conceptId"])

rels_after_merge = snomed_rels.shape
print(f"\u2192 Merge completed: {rels_before_merge} \u2192 {rels_after_merge}")

print("\n Cleaning data...")
before_clean = len(snomed_rels)

# Drop empty fields
snomed_rels = snomed_rels.dropna(subset=["source_term", "destination_term", "relation_term"])
after_dropna = len(snomed_rels)
print(f"Removed empty rows: {before_clean - after_dropna:,}")

# Drop rows with blank strings
mask_blank = ((snomed_rels["source_term"].str.strip() == "") |  (snomed_rels["destination_term"].str.strip() == ""))
removed_blank = mask_blank.sum()
snomed_rels = snomed_rels[~mask_blank]
print(f"Removed blank-term rows: {removed_blank:,}")


# Reorder columns
snomed_rels = snomed_rels[["sourceId", "source_term", "relation_term", "destinationId", "destination_term"]]

#Save final dataset
out_path = "snomed_relations_full.tsv"
snomed_rels.to_csv(out_path, sep="\t", index=False)

#Summary report
print("\n====== SUMMARY REPORT ======")
print(f"Final dataset shape: {snomed_rels.shape}")
print(f"Saved to: {out_path}")
print(snomed_rels.sample(min(5, len(snomed_rels)), random_state=42).to_string(index=False))



 Loading SNOMED CT raw files...
→ Relationships: (3581598, 10), Descriptions: (1696627, 9)
 Active filter: 3,581,598 → 1,336,381 (removed 2,245,217)
 Preferred terms: 1,696,627 → 994,002 (removed 702,625)
 Found 101 unique SNOMED relation types

 Merging concept names...
→ Merge completed: (1336381, 11) → (7161453, 13)

 Cleaning data...
Removed empty rows: 1
Removed blank-term rows: 0

====== SUMMARY REPORT ======
Final dataset shape: (7161452, 5)
Saved to: snomed_relations_full.tsv
  sourceId                                                               source_term   relation_term destinationId      destination_term
1231766002                                     Hemorrhage after thrombolytic therapy            Is a     131148009           Haemorrhage
  27520001                                 Pustular psoriasis of the palms and soles      Morphology      48055004         Pustular rash
 766940004                                                                      Role            Is 

In [ ]:
import pandas as pd
from itertools import combinations
import zipfile # Import zipfile
import os      # Import os
from tqdm import tqdm # For progress bar

# 1. Load and Clean MIMIC-IV Diagnoses
mimic_path = os.path.join(BASE_DRIVE_PATH, "/content/drive/MyDrive/knowledgegraphdata/diagnoses_icd.csv.gz")
print("Loading MIMIC-IV diagnoses data...")
diagnoses = pd.read_csv(mimic_path, dtype=str)
print(f"Loaded: {diagnoses.shape[0]:,} rows \u00d7 {diagnoses.shape[1]} columns\n")

cols_to_keep = ["subject_id", "hadm_id", "seq_num", "icd_code", "icd_version"]
diagnoses = diagnoses[cols_to_keep]

before_filter = len(diagnoses)
diagnoses = diagnoses[diagnoses["icd_version"] == "10"]
after_filter = len(diagnoses)
print(f"ICD-10 filter applied: {before_filter:,} \u2192 {after_filter:,} rows")

before_clean = len(diagnoses)
diagnoses = diagnoses.dropna(subset=["icd_code"])
diagnoses = diagnoses[diagnoses["icd_code"].str.strip() != ""]
after_clean = len(diagnoses)
print(f"Cleaned empty ICD codes: {before_clean:,} \u2192 {after_clean:,} rows")

before_dedup = len(diagnoses)
diagnoses.drop_duplicates(subset=["subject_id", "hadm_id", "icd_code"], inplace=True)
after_dedup = len(diagnoses)
print(f"Removed duplicates: {before_dedup - after_dedup:,}")

diagnoses.reset_index(drop=True, inplace=True)

out_path = "mimic_icd10_clean.csv"
diagnoses.to_csv(out_path, index=False)
print(f"\nCleaned ICD-10 data saved \u2192 {out_path}")

print("\nSample of cleaned dataset:")
print(diagnoses.sample(min(10, len(diagnoses)), random_state=42).to_string(index=False))

# 2. Extract ICD \u2192 SNOMED CT Mapping from Athena
print("\nLoading OHDSI Athena vocabularies...")

# --- Load CSVs directly from ZIPs into memory ---
zip_file_concept = os.path.join(BASE_DRIVE_PATH, "/content/drive/MyDrive/knowledgegraphdata/CONCEPT.zip")
with zipfile.ZipFile(zip_file_concept, 'r') as zf:
    with zf.open('CONCEPT.csv') as f:
        concepts = pd.read_csv(f, sep="\t", dtype=str, low_memory=False)
print(f"Loaded CONCEPT.csv from {zip_file_concept}")

zip_file_rel = os.path.join(BASE_DRIVE_PATH, "/content/drive/MyDrive/knowledgegraphdata/CONCEPT_RELATIONSHIP.zip")
with zipfile.ZipFile(zip_file_rel, 'r') as zf:
    with zf.open('CONCEPT_RELATIONSHIP.csv') as f:
        rels = pd.read_csv(f, sep="\t", dtype=str, low_memory=False)
print(f"Loaded CONCEPT_RELATIONSHIP.csv from {zip_file_rel}")
# ---------------------------------------------------

# Ensure vocabulary_id is clean right after loading
concepts['vocabulary_id'] = concepts['vocabulary_id'].str.strip()

# Ensure relationship_id is clean right after loading
rels['relationship_id'] = rels['relationship_id'].str.strip()

print(f"Concepts: {len(concepts):,} rows, Relationships: {len(rels):,} rows")
print("\n--- Debug: concepts DataFrame head ---")
display(concepts.head())
print("\n--- Debug: rels DataFrame head ---")
display(rels.head())

mapping = rels[rels["relationship_id"].isin(["Maps to", "Maps to value"])]

concepts_small = concepts[["concept_id", "concept_code", "concept_name", "vocabulary_id"]]
merged = (
    mapping
    .merge(concepts_small, left_on="concept_id_1", right_on="concept_id")
    .merge(concepts_small, left_on="concept_id_2", right_on="concept_id", suffixes=('_source', '_destination'))
)
print("\n--- Debug: merged DataFrame head (relevant columns) ---")
display(merged[["concept_code_source", "vocabulary_id_source", "concept_id_2", "concept_name_destination", "vocabulary_id_destination"]].head())

# 3. Create MIMIC-to-SNOMED mapping
print("\nCreating MIMIC-to-SNOMED mapping...")
# Filter mapping for ICD10CM source and SNOMED destination
icd_snomed_map_df = merged[
    (merged["vocabulary_id_source"] == "ICD10CM") &
    (merged["vocabulary_id_destination"] == "SNOMED")
]
# Select and rename columns for clarity
icd_snomed_map_df = icd_snomed_map_df[[
    "concept_code_source", "concept_id_2", "concept_name_destination"
]].rename(columns={
    "concept_code_source": "icd_code",
    "concept_id_2": "snomed_concept_id",
    "concept_name_destination": "snomed_concept_name"
}).drop_duplicates(subset=["icd_code", "snomed_concept_id"])
print(f"ICD10CM to SNOMED mapping entries: {len(icd_snomed_map_df):,}")

# --- Debugging print statements ---
print(f"ICD codes in diagnoses: {diagnoses['icd_code'].unique()}")
print(f"Is 'A000' in icd_snomed_map_df? {'A000' in icd_snomed_map_df['icd_code'].values}")
if 'A000' in icd_snomed_map_df['icd_code'].values:
    print("Found 'A000' in icd_snomed_map_df. Sample entry:")
    print(icd_snomed_map_df[icd_snomed_map_df['icd_code'] == 'A000'].head().to_string(index=False))
else:
    print(" 'A000' not found in icd_snomed_map_df.")
# -----------------------------------

# Merge MIMIC diagnoses with SNOMED concepts
diagnoses_snomed = diagnoses.merge(
    icd_snomed_map_df,
    on="icd_code",
    how="inner" # Keep only diagnoses that can be mapped to SNOMED
)
print(f"MIMIC diagnoses after SNOMED mapping: {len(diagnoses_snomed):,} rows")
diagnoses_snomed.dropna(subset=["snomed_concept_id"], inplace=True)
diagnoses_snomed.drop_duplicates(subset=["subject_id", "hadm_id", "snomed_concept_id"], inplace=True)
print(f"MIMIC diagnoses (unique SNOMED per admission): {len(diagnoses_snomed):,} rows")


# 4. Extract Co-occurrence Pairs
print("\nExtracting co-occurrence pairs...")
co_occurrence_list = []
# Group by admission and iterate
for (subject_id, hadm_id), group in tqdm(diagnoses_snomed.groupby(["subject_id", "hadm_id"]), desc="Processing admissions"):
    snomed_codes = sorted(group["snomed_concept_id"].unique().tolist())
    snomed_names = {row["snomed_concept_id"]: row["snomed_concept_name"] for idx, row in group.iterrows()}

    # Generate all unique pairs of SNOMED codes within this admission
    for code1, code2 in combinations(snomed_codes, 2):
        # Ensure consistent order for pairs
        if code1 < code2:
            co_occurrence_list.append({
                "sourceId": code1,
                "source_term": snomed_names.get(code1, "UNKNOWN"),
                "relation_term": "co_occurs_with",
                "destinationId": code2,
                "destination_term": snomed_names.get(code2, "UNKNOWN")
            })
        else:
             co_occurrence_list.append({
                "sourceId": code2,
                "source_term": snomed_names.get(code2, "UNKNOWN"),
                "relation_term": "co_occurs_with",
                "destinationId": code1,
                "destination_term": snomed_names.get(code1, "UNKNOWN")
            })


# Create DataFrame from co-occurrence list
mimic_snomed_pairs = pd.DataFrame(co_occurrence_list)

# If no pairs are generated (e.g., due to limited placeholder data), create an empty df with correct columns
if mimic_snomed_pairs.empty:
    mimic_snomed_pairs = pd.DataFrame(columns=[
        "sourceId", "source_term", "relation_term", "destinationId", "destination_term"
    ])


print(f"Generated {len(mimic_snomed_pairs):,} co-occurrence pairs.")

# 5. Save the co-occurrence pairs
mimic_snomed_pairs_out_path = "mimic_snomed_pairs.tsv"
mimic_snomed_pairs.to_csv(mimic_snomed_pairs_out_path, sep="\t", index=False)
print(f"MIMIC co-occurrence pairs saved to \u2192 {mimic_snomed_pairs_out_path}")

Loading MIMIC-IV diagnoses data...
Loaded: 6,364,488 rows × 5 columns

ICD-10 filter applied: 6,364,488 → 3,455,747 rows
Cleaned empty ICD codes: 3,455,747 → 3,455,747 rows
Removed duplicates: 58

Cleaned ICD-10 data saved → mimic_icd10_clean.csv

Sample of cleaned dataset:
subject_id  hadm_id seq_num icd_code icd_version
  11979534 27415150       5     N185          10
  11130556 28389443       8   F10239          10
  16475227 21680259      14     M353          10
  19645775 21606532       8    J9811          10
  12448853 22437295       2    K5289          10
  13847788 27562694      10  T17590A          10
  19368574 29840211       7     M109          10
  12833439 22586352      12   Z87891          10
  19655369 27157385       8    Z9884          10
  15221512 29743103      11    I5021          10

Loading OHDSI Athena vocabularies...
Loaded CONCEPT.csv from /content/drive/MyDrive/knowledgegraphdata/CONCEPT.zip
Loaded CONCEPT_RELATIONSHIP.csv from /content/drive/MyDrive/knowledgeg

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,45756805,Pediatric Cardiology,Provider,ABMS,Physician Specialty,S,OMOP4821938,19700101,20991231,NaN
1,45756804,Pediatric Anesthesiology,Provider,ABMS,Physician Specialty,S,OMOP4821939,19700101,20991231,NaN
2,45756803,Pathology-Anatomic / Pathology-Clinical,Provider,ABMS,Physician Specialty,S,OMOP4821940,19700101,20991231,NaN
3,45756802,Pathology - Pediatric,Provider,ABMS,Physician Specialty,S,OMOP4821941,19700101,20991231,NaN
4,45756801,Pathology - Molecular Genetic,Provider,ABMS,Physician Specialty,S,OMOP4821942,19700101,20991231,NaN



--- Debug: rels DataFrame head ---


,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
0,19082573,36935620,RxNorm dose form of,20230522,20991231,NaN
1,40703384,36962214,Has marketed form,20230522,20991231,NaN
2,19082573,36939166,RxNorm dose form of,20230522,20991231,NaN
3,19001949,36946096,RxNorm dose form of,20230522,20991231,NaN
4,19082573,36942470,RxNorm dose form of,20230522,20991231,NaN



--- Debug: merged DataFrame head (relevant columns) ---


,concept_code_source,vocabulary_id_source,concept_id_2,concept_name_destination,vocabulary_id_destination
0,OMOP5199592,RxNorm Extension,36919314,Tozinameran 0.05 MG/ML,RxNorm Extension
1,OMOP5212221,RxNorm Extension,36919391,100 ML amoxicillin 25 MG/ML / clavulanate 6.25...,RxNorm Extension
2,OMOP5208019,RxNorm Extension,36919422,0.8 ML adalimumab 50 MG/ML Injectable Solution...,RxNorm Extension
3,OMOP5185964,RxNorm Extension,36919439,ondansetron 8 MG Disintegrating Oral Tablet by...,RxNorm Extension
4,OMOP5191160,RxNorm Extension,36919440,tadalafil 10 MG Delayed Release Oral Tablet Bo...,RxNorm Extension



Creating MIMIC-to-SNOMED mapping...
ICD10CM to SNOMED mapping entries: 129,908
ICD codes in diagnoses: ['G3183' 'F0280' 'R441' ... 'O30093' 'V835XXA' 'O359XX2']
Is 'A000' in icd_snomed_map_df? False
 'A000' not found in icd_snomed_map_df.
MIMIC diagnoses after SNOMED mapping: 218,769 rows
MIMIC diagnoses (unique SNOMED per admission): 218,768 rows

Extracting co-occurrence pairs...


Processing admissions: 100%|██████████| 148363/148363 [00:23<00:00, 6338.22it/s]


Generated 97,033 co-occurrence pairs.
MIMIC co-occurrence pairs saved to → mimic_snomed_pairs.tsv


In [ ]:
# Cell 5: Merge SNOMED and MIMIC (from merge_snomed_and_mimic.py)

import pandas as pd

SNOMED_PATH = "snomed_relations_full.tsv"
MIMIC_PATH  = "mimic_snomed_pairs.tsv"
OUT_PATH    = "merged_relations.tsv"

print("Loading input files...")
snomed_df = pd.read_csv(SNOMED_PATH, sep="\t", dtype=str)
mimic_df  = pd.read_csv(MIMIC_PATH, sep="\t", dtype=str)
print(f"SNOMED relations: {len(snomed_df):,}")
print(f"MIMIC co-occurrences: {len(mimic_df):,}\n")

# === Normalize columns for consistency ===
required_cols = ["sourceId", "source_term", "relation_term", "destinationId", "destination_term"]

def ensure_cols(df, name):
    missing = set(required_cols) - set(df.columns)
    if missing:
        raise ValueError(f"{name} missing columns: {missing}")
    return df[required_cols].copy()

snomed_df = ensure_cols(snomed_df, "SNOMED")
mimic_df  = ensure_cols(mimic_df, "MIMIC")

# === Add dataset source label (optional) ===
snomed_df["source"] = "SNOMED_CT"
mimic_df["source"]  = "MIMIC_IV"

# === Merge ===
print("Merging datasets...")
before_merge_snomed = len(snomed_df)
before_merge_mimic  = len(mimic_df)
merged = pd.concat([snomed_df, mimic_df], ignore_index=True)

after_merge = len(merged)
print(f"Before merge: SNOMED={before_merge_snomed:,}, MIMIC={before_merge_mimic:,}")
print(f"After merge (combined): {after_merge:,}")

# === Remove duplicates ===
before_clean = len(merged)
merged.drop_duplicates(subset=["sourceId", "relation_term", "destinationId"], inplace=True)
after_clean = len(merged)
print(f"Duplicate removal: {before_clean:,} → {after_clean:,} (removed {before_clean - after_clean:,})")

# === Summary ===
print("\nSaving merged graph → {OUT_PATH}")
merged.to_csv(OUT_PATH, sep="\t", index=False)
print(f"Merged relational graph saved: {len(merged):,} rows")

Loading input files...
SNOMED relations: 7,161,452
MIMIC co-occurrences: 97,033

Merging datasets...
Before merge: SNOMED=7,161,452, MIMIC=97,033
After merge (combined): 7,258,485
Duplicate removal: 7,258,485 → 1,299,286 (removed 5,959,199)

Saving merged graph → {OUT_PATH}
Merged relational graph saved: 1,299,286 rows


In [ ]:
# Cell 6: Graph Tensor Creation and Splitting (from Dataset_creation.py)

import pandas as pd
import numpy as np
import torch
from collections import defaultdict

IN_PATH = "merged_relations.tsv"
OUT_GRAPH = "graph_data.pt"
OUT_NODEMAP = "node_id_map.csv"
OUT_RELMAP = "rel_id_map.csv"

SPLIT_RATIO = (0.8, 0.1, 0.1)   # train/val/test
RANDOM_SEED = 42

print(f"Loading merged dataset: {IN_PATH}")
df = pd.read_csv(IN_PATH, sep="\t", dtype=str)
print(f"Loaded: {len(df):,} edges, {df['relation_term'].nunique():,} relation types")

required_cols = ["sourceId", "relation_term", "destinationId"]
for c in required_cols:
    if c not in df.columns:
        raise ValueError(f"Missing column: {c}")

# 1. Clean and validate (redundant due to merge_snomed_and_mimic.py but kept for robustness)
df = df.dropna(subset=required_cols)
df = df[(df["sourceId"].str.strip() != "") & (df["destinationId"].str.strip() != "")]
df.reset_index(drop=True, inplace=True)

# 2. Create node index mapping
nodes = pd.Index(pd.unique(pd.concat([df["sourceId"], df["destinationId"]], ignore_index=True)))
node2id = {nid: i for i, nid in enumerate(nodes)}
print(f"Unique nodes: {len(node2id):,}")

# 3. Create relation index mapping
base_relations = sorted(df["relation_term"].unique().tolist())
rel2id = {r: i for i, r in enumerate(base_relations)}
print(f"Base relation types: {len(rel2id)}")

# Encode edges
src = df["sourceId"].map(node2id).astype(np.int64).values
dst = df["destinationId"].map(node2id).astype(np.int64).values
rel = df["relation_term"].map(rel2id).astype(np.int64).values

# 4. Add inverse relations
inv_relations = [r + "_inv" for r in base_relations]
inv_offset = len(rel2id)
for i, r in enumerate(inv_relations):
    rel2id[r] = inv_offset + i

src_inv = dst.copy()
dst_inv = src.copy()
rel_inv = rel + inv_offset

# Concatenate original + inverse
src_all = np.concatenate([src, src_inv])
dst_all = np.concatenate([dst, dst_inv])
rel_all = np.concatenate([rel, rel_inv])

num_nodes = len(node2id)
num_relations = len(rel2id)
print(f"Total relations (including inverses): {num_relations}")
print(f"Total edges after adding inverses: {len(src_all):,}")

# 5. Build PyTorch tensors
edge_index = torch.tensor(np.vstack([src_all, dst_all]), dtype=torch.long)
edge_type = torch.tensor(rel_all, dtype=torch.long)

# 6. Train/Val/Test Split
rng = np.random.default_rng(RANDOM_SEED)
E = len(src)  # original (non-inverse) edges
perm = rng.permutation(E)

n_train = int(SPLIT_RATIO[0] * E)
n_val = int(SPLIT_RATIO[1] * E)

train_idx_base = perm[:n_train]
val_idx_base = perm[n_train:n_train + n_val]
test_idx_base = perm[n_train + n_val:]

# Duplicate indices for inverse edges
train_idx = np.concatenate([train_idx_base, train_idx_base + E])
val_idx = np.concatenate([val_idx_base, val_idx_base + E])
test_idx = np.concatenate([test_idx_base, test_idx_base + E])

train_idx = torch.tensor(train_idx, dtype=torch.long)
val_idx = torch.tensor(val_idx, dtype=torch.long)
test_idx = torch.tensor(test_idx, dtype=torch.long)

# 7. Save all outputs
graph_data = {
    "num_nodes": num_nodes,
    "num_relations": num_relations,
    "edge_index": edge_index,
    "edge_type": edge_type,
    "train_idx": train_idx,
    "val_idx": val_idx,
    "test_idx": test_idx,
    "node_ids": list(nodes),
    "rel2id": rel2id,
    "base_relations": base_relations,
}

torch.save(graph_data, OUT_GRAPH)
print(f"Saved graph tensors → {OUT_GRAPH}")

# Save mappings for interpretability
pd.DataFrame({"node_index": np.arange(num_nodes), "concept_id": list(nodes)}).to_csv(OUT_NODEMAP, index=False)
pd.DataFrame({"relation": list(rel2id.keys()), "rel_id": list(rel2id.values())}).to_csv(OUT_RELMAP, index=False)
print(f"Saved node map → {OUT_NODEMAP}")
print(f"Saved relation map → {OUT_RELMAP}")

print("\nGraph construction complete.")
print(f"Nodes: {num_nodes:,} | Relations: {num_relations:,} | Edges: {len(src_all):,}")
print(f"Train edges: {len(train_idx)}, Val: {len(val_idx)}, Test: {len(test_idx)}")

Loading merged dataset: merged_relations.tsv
Loaded: 1,299,286 edges, 102 relation types
Unique nodes: 382,356
Base relation types: 102
Total relations (including inverses): 204
Total edges after adding inverses: 2,598,572
Saved graph tensors → graph_data.pt
Saved node map → node_id_map.csv
Saved relation map → rel_id_map.csv

Graph construction complete.
Nodes: 382,356 | Relations: 204 | Edges: 2,598,572
Train edges: 2078856, Val: 259856, Test: 259860


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.nn import TransE
from tqdm import tqdm
import os
import pandas as pd
import numpy as np

# --- 1. Custom Classifier Module ---
class AdaptedTransE(nn.Module):
    def __init__(self, num_nodes, num_relations, embedding_dim, dropout_rate=0.3):
        super().__init__()
        self.num_nodes = num_nodes
        self.num_relations = num_relations
        self.embedding_dim = embedding_dim

        # 1. Embedding Layer (TransE from PyG, for learning embeddings h and t)
        # Note: We use this layer purely for its learned entity embeddings and batching utility.
        # The TransE relation embeddings (e_r) are not used in the final classifier.
        self.transe = TransE(
            num_nodes=num_nodes,
            num_relations=num_relations,
            hidden_channels=embedding_dim,
            margin=1.0,
            p_norm=1.0,
        )

        # 2. Relation Classifier (Multi-Head Classifier as described in the paper)
        # It takes the difference vector (h - t), which is a tensor of size [batch_size, embedding_dim].
        # It maps this vector to the number of relation types.
        self.classifier = nn.Sequential(
            # First, normalize or use dropout on the difference vector (h - t)
            nn.Dropout(p=dropout_rate),
            # Linear layer to map the difference vector to logits for each relation type
            nn.Linear(embedding_dim, num_relations)
        )

        # Initialize the classifier's weights (using Xavier for stability, as suggested for embeddings)
        nn.init.xavier_uniform_(self.classifier[1].weight)

    def forward(self, head_index, tail_index):
        # 1. Get Normalized Entity Embeddings from the TransE layer
        # The TransE model internally maintains normalized entity embeddings.
        h = self.transe.node_emb(head_index)
        t = self.transe.node_emb(tail_index)

        # 2. Calculate the difference vector (h - t) which approximates the relation r
        diff_vector = h - t

        # 3. Pass the difference vector to the classifier head
        logits = self.classifier(diff_vector)

        return logits

    # We will use the standard PyG TransE loss for pre-training the embeddings
    def transe_loss(self, head_index, rel_type, tail_index):
        return self.transe.loss(head_index, rel_type, tail_index)

# --- 2. Training Function for the Classifier ---
def train_classifier(model, optimizer, data_loader, edge_index, edge_type, base_relations, num_epochs):
    model.train()

    # We need to map the full edge_type tensor to only the base relations (non-inverse)
    # to match the classifier's output size.
    # We assume base relations are the first half of the total relations (as per Dataset_creation.py)
    num_base_relations = len(base_relations)

    # We only train on the forward (non-inverse) relationships.
    forward_mask = edge_type < num_base_relations
    forward_indices = torch.where(forward_mask)[0]

    # Use only the training set indices for the forward relations
    # `graph_data['train_idx']` is now guaranteed to be on the correct device
    train_indices_forward = forward_indices[torch.isin(forward_indices, graph_data['train_idx'])]

    # Create a new loader that iterates over the forward-only triples
    classifier_loader = model.transe.loader(
        head_index=edge_index[0, train_indices_forward],
        rel_type=edge_type[train_indices_forward],
        tail_index=edge_index[1, train_indices_forward],
        batch_size=BATCH_SIZE,
        shuffle=True,
    )

    print("\nStarting Adapted TransE Classifier Training...")

    for epoch in range(1, num_epochs + 1):
        total_loss = 0
        correct_predictions = 0
        total_samples = 0

        for head_index, rel_type, tail_index in classifier_loader:
            optimizer.zero_grad()

            # Predict the relation type using the classifier
            logits = model(head_index, tail_index)

            # Use Cross-Entropy Loss for multi-class classification
            loss = F.cross_entropy(logits, rel_type)

            loss.backward()
            optimizer.step()

            total_loss += loss.item() * head_index.size(0)
            total_samples += head_index.size(0)

            # Calculate accuracy
            _, predicted = torch.max(logits, 1)
            correct_predictions += (predicted == rel_type).sum().item()

        avg_loss = total_loss / total_samples
        accuracy = correct_predictions / total_samples

        print(f"Epoch: {epoch:03d}, Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.4f}")

        if epoch % 5 == 0 or epoch == num_epochs:
            val_accuracy = evaluate_classifier(model, edge_index, edge_type, graph_data['val_idx'], num_base_relations)
            print(f"  >>> Validation Accuracy: {val_accuracy:.4f}")

    # Final Test
    test_accuracy = evaluate_classifier(model, edge_index, edge_type, graph_data['test_idx'], num_base_relations)
    print("\n--- FINAL CLASSIFIER TEST RESULTS ---")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print("-------------------------------------")


@torch.no_grad()
def evaluate_classifier(model, edge_index, edge_type, eval_idx, num_base_relations):
    model.eval()

    # Select only the forward (non-inverse) relations in the evaluation set
    eval_mask = edge_type < num_base_relations
    # `eval_idx` is now guaranteed to be on the correct device
    eval_indices_forward = eval_idx[torch.isin(eval_idx, torch.where(eval_mask)[0])]

    if eval_indices_forward.numel() == 0:
        return 0.0

    head_index = edge_index[0, eval_indices_forward]
    tail_index = edge_index[1, eval_indices_forward]
    rel_type = edge_type[eval_indices_forward]

    logits = model(head_index, tail_index)
    _, predicted = torch.max(logits, 1)

    correct_predictions = (predicted == rel_type).sum().item()
    total_samples = rel_type.size(0)

    return correct_predictions / total_samples

# --- 3. Main Execution (TransE Pre-training + Classifier Training) ---
# --- Configuration (using paper's optimal values for the classifier part) ---
EMBEDDING_DIM = 200      # Based on best run
CLASSIFIER_LR = 0.001    # Learning rate for Adam
CLASSIFIER_EPOCHS = 10   # Based on paper's finding
PRE_TRAIN_EPOCHS = 50    # Pre-train longer than 10 for better embeddings
BATCH_SIZE = 4096         # Based on best run

GRAPH_DATA_PATH = "graph_data.pt"
if not os.path.exists(GRAPH_DATA_PATH):
    print("WARNING: graph_data.pt not found. Running placeholder data preparation now...")
    # This block requires running the data prep cells (1-6) first.
    # Since they were provided in the prompt, we assume they have run or will run.
    raise FileNotFoundError("Please ensure you run all cells prior to Cell 7, including the data preparation.")

graph_data = torch.load(GRAPH_DATA_PATH)

# --- Prepare Tensors and Device ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

num_nodes = graph_data["num_nodes"]
num_relations = graph_data["num_relations"]
base_relations = graph_data["base_relations"]

# Move all relevant tensors to the device
graph_data["edge_index"] = graph_data["edge_index"].to(device)
graph_data["edge_type"] = graph_data["edge_type"].to(device)
graph_data["train_idx"] = graph_data["train_idx"].to(device)
graph_data["val_idx"] = graph_data["val_idx"].to(device)
graph_data["test_idx"] = graph_data["test_idx"].to(device)

edge_index = graph_data["edge_index"]
edge_type = graph_data["edge_type"]

# --- Initialize Model ---
model = AdaptedTransE(num_nodes, num_relations, EMBEDDING_DIM).to(device)

# --- PHASE 1: TransE Pre-training (Optimizing embeddings for link prediction) ---
print("\n--- PHASE 1: TransE Embedding Pre-training ---")
pre_train_optimizer = optim.Adam(model.transe.parameters(), lr=0.01) # Using standard KGE learning rate
transe_loader = model.transe.loader(
    head_index=edge_index[0],
    rel_type=edge_type,
    tail_index=edge_index[1],
    batch_size=BATCH_SIZE,
    shuffle=True,
)

for epoch in tqdm(range(1, PRE_TRAIN_EPOCHS + 1), desc="TransE Pre-training"):
    total_loss = 0
    for head_index, rel_type, tail_index in transe_loader:
        pre_train_optimizer.zero_grad()
        loss = model.transe_loss(head_index, rel_type, tail_index)
        loss.backward()
        pre_train_optimizer.step()
        total_loss += float(loss) * head_index.numel()

    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d} Loss: {total_loss / edge_index.size(1):.4f}")

# --- PHASE 2: Relation Classification (Optimizing classifier on fixed embeddings) ---
print("\n--- PHASE 2: Relation Classification Training ---")

# Fix TransE embeddings and optimize only the classifier weights
for param in model.transe.parameters():
    param.requires_grad = False

classifier_optimizer = optim.Adam(model.classifier.parameters(), lr=CLASSIFIER_LR)

# Train the classifier using the forward-only data subset
train_classifier(model, classifier_optimizer, None, edge_index, edge_type, base_relations, CLASSIFIER_EPOCHS)


Using device: cpu

--- PHASE 1: TransE Embedding Pre-training ---


TransE Pre-training:   0%|          | 0/50 [00:00<?, ?it/s]/tmp/ipython-input-4158809868.py:209: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:836.)
  total_loss += float(loss) * head_index.numel()
TransE Pre-training:   8%|▊         | 4/50 [55:56<10:43:16, 839.05s/it]


KeyboardInterrupt: 

# Task
## Explain Hits@10 and Link Prediction

**Hits@10** is a metric commonly used in **link prediction** tasks for knowledge graphs. It measures how often the correct entity is ranked among the top `k` (e.g., 10) candidates when a missing entity in a triple is being predicted.

In link prediction, the goal is to predict either the head entity (h), the relation (r), or the tail entity (t) of a triple `(h, r, t)` when one component is unknown. For example, given `(h, r, ?)` (predicting the tail entity), the model generates scores for all possible tail entities. Hits@10 then checks if the *true* tail entity for that triple is found among the top 10 highest-scoring candidates. A higher Hits@10 indicates better link prediction performance.

**How it differs from classification accuracy:**

The **classification accuracy** reported in the previous step (96.69%) was for **relation classification**. In that task, given a pair of head and tail entities `(h, ?, t)`, the model predicts the *type* of relation `r` that connects them. It's a multi-class classification problem where the model outputs probabilities (logits) for each possible relation type, and the accuracy is computed based on whether the highest-probability relation matches the true relation.

In contrast, **link prediction** (and thus Hits@10) focuses on predicting *entities*, not relation types. It assesses how well the learned embeddings capture the structural relationships in the knowledge graph, allowing the model to "fill in the blanks" for missing entities. Both are important for evaluating knowledge graph embeddings but measure different aspects of the model's understanding of the graph.

## Link Prediction Evaluation

Now, let's implement and evaluate the Hits@10 metric for the pre-trained TransE embeddings.

```python
# Cell 8: Link Prediction Evaluation

import torch
import torch.nn.functional as F
from tqdm import tqdm

# It's good practice to redefine the function if the agent restarts or if we want to run this cell independently.
# However, for continuation, we can assume `model`, `graph_data`, `device` are available from the previous cell.

@torch.no_grad()
def evaluate_link_prediction(transe_model, edge_index, edge_type, eval_indices, k_values=[1, 3, 10], num_negatives=100):
    """
    Evaluates link prediction performance using Hits@k and Mean Rank.

    Args:
        transe_model: The TransE model (e.g., model.transe) with trained embeddings.
        edge_index: Tensor of shape [2, num_edges] representing head and tail indices.
        edge_type: Tensor of shape [num_edges] representing relation types.
        eval_indices: Indices for the evaluation set (e.g., test_idx).
        k_values: List of k values for Hits@k.
        num_negatives: Number of negative samples to generate for each positive triple.
    Returns:
        A dictionary containing Hits@k and Mean Rank metrics.
    """
    transe_model.eval()

    # Extract head, relation, tail for the evaluation triples
    head_indices = edge_index[0, eval_indices]
    rel_types = edge_type[eval_indices]
    tail_indices = edge_index[1, eval_indices]

    results = {}
    print(f"\nPerforming link prediction evaluation on {len(eval_indices)} triples with {num_negatives} negative samples per triple...")
    
    # PyG's k_hit function internally computes mean rank and Hits@k by corrupting
    # both head and tail entities and averaging the results.
    for k in k_values:
        hits_at_k, mean_rank = transe_model.k_hit(
            head_index=head_indices,
            rel_type=rel_types,
            tail_index=tail_indices,
            k=k,
            num_negatives=num_negatives,
            batch_size=4096 # Use batching for k_hit to manage memory
        )
        results[f"Hits@{k}"] = hits_at_k
        print(f"  Hits@{k}: {hits_at_k:.4f}")
    
    # Mean Rank is computed once for all k values by k_hit. We can just take one.
    results["Mean Rank"] = mean_rank
    print(f"  Mean Rank: {mean_rank:.4f}")
    return results

# --- Call Link Prediction Evaluation after TransE Pre-training ---
# The 'model' object contains the pre-trained TransE embeddings (model.transe)
# from the previous training phase.

print("\n--- Evaluating Link Prediction on Test Set (after TransE Pre-training) ---")
link_pred_metrics = evaluate_link_prediction(
    model.transe,
    graph_data["edge_index"],
    graph_data["edge_type"],
    graph_data["test_idx"], # Evaluate on the test set
    k_values=[1, 3, 10]
)
print("-------------------------------------------------------")
print("Final Link Prediction Metrics (Test Set):")
for metric, value in link_pred_metrics.items():
    print(f"  {metric}: {value:.4f}")
print("-------------------------------------------------------")
```

## Explain Hits@10 and Link Prediction

### Subtask:
Clarify what Hits@10 means in the context of knowledge graph embeddings and link prediction, and why it's different from the classification accuracy reported.


### Clarifying Hits@10 and Link Prediction vs. Classification Accuracy

#### What is Link Prediction?

Link prediction is a common task in knowledge graph analysis. It aims to predict missing links (relationships) between entities (nodes) in a knowledge graph. Given a triplet `(head_entity, relation, tail_entity)`, the goal is to:

1.  **Predict a missing tail entity**: Given `(head_entity, relation, ?)`
2.  **Predict a missing head entity**: Given `(?, relation, tail_entity)`
3.  **(Less common for evaluation)** Predict a missing relation: Given `(head_entity, ?, tail_entity)`

Link prediction models learn to embed entities and relations into a continuous vector space, such that the geometric relationships between these embeddings reflect the semantic relationships in the graph. For instance, in TransE-like models, if `(h, r, t)` is true, then `h + r ≈ t` in the embedding space.

#### What is Hits@K?

Hits@K is a common metric used to evaluate the performance of link prediction models. Specifically, it measures the proportion of *correctly predicted entities* (either head or tail) that appear within the top K ranked predictions. Here's how it works:

1.  **Corrupting Triples**: For each true triple `(h, r, t)` in the test set, two corrupted versions are generated:
    *   `(h, r, t')`: Replace the true tail `t` with a randomly chosen *incorrect* tail entity `t'`. This is repeated for many `t'` candidates.
    *   `(h', r, t)`: Replace the true head `h` with a randomly chosen *incorrect* head entity `h'`. This is repeated for many `h'` candidates.

2.  **Scoring and Ranking**: The link prediction model then scores how likely each corrupted triple is to be true. For example, in `(h, r, ?)`, the model would score `(h, r, e_i)` for *all* possible entities `e_i` in the graph. These scores are then ranked from most likely to least likely.

3.  **Calculating Hits@K**: If the *true* missing entity (e.g., `t` in `(h, r, ?)`) is found among the top K highest-scoring predictions, it counts as a "hit".

    *   **Hits@10** means that if the correct entity is found within the top 10 ranked predictions, it's counted as a hit. A higher Hits@10 score indicates a better model because it means the correct answer is frequently among the top plausible choices.

#### How is Hits@10 Different from Classification Accuracy?

The previous output from Cell 7 provided **Training Accuracy** and **Test Accuracy** for a **relation classification task**, which focused on predicting the *relation type* given a head and tail entity (or more specifically, the difference vector between head and tail embeddings).

Here's the key distinction:

*   **Relation Classification Accuracy**: Measures how well the model can identify the *correct relation type* `r` given a specific `(h, ?, t)` pair. The model predicts one of `num_base_relations` categories, and accuracy is simply `(correct_predictions / total_predictions)`. The output of the classifier is a probability distribution over relation types.

*   **Hits@10 (Link Prediction)**: Measures how well the model can identify the *correct missing entity* (head or tail) given an `(h, r, ?)` or `(?, r, t)` pair. This is a much harder task, as the model has to rank the correct entity among *all possible entities* (hundreds of thousands in our case, `num_nodes`). The output is not a single classification, but a ranked list of entities.

**Analogy**:

*   **Relation Classification** is like being shown two people and asked, "What is their relationship? (e.g., 'parent of', 'sibling of', 'friend of')". You choose from a fixed, relatively small list of options.
*   **Link Prediction with Hits@10** is like being told, "This person is the 'parent of' someone. Who is it?" and you have to pick the correct child from everyone in the world. Hits@10 asks if the correct child was among your top 10 guesses.

**Why the difference in reported metrics?**

*   The previous code built a **classifier** on top of the learned embeddings specifically to predict the `relation_term` given `(h, t)`. Its accuracy directly reflects how well it performs this *classification* task.
*   To evaluate link prediction (predicting missing entities), one would typically use metrics like Hits@K, Mean Rank (MR), and Mean Reciprocal Rank (MRR). The current code *does not* perform a link prediction evaluation, so these metrics are not reported. The model's `transe.forward()` method is designed for scoring triples for link prediction, but the current training loop and evaluation functions are tailored for relation classification.

## Implement Link Prediction Evaluation Function

### Subtask:
Define the `evaluate_link_prediction` function to calculate Hits@k and Mean Rank.


**Reasoning**:
To implement the link prediction evaluation function, I will define a Python function `evaluate_link_prediction` that takes the model, data, and a list of `k` values as input, and calculates Hits@k and Mean Rank. This function will utilize the `tqdm` library for progress indication and ensure computations are performed on the correct device (CPU/GPU).



In [ ]:
from tqdm import tqdm

def evaluate_link_prediction(model, head_index, rel_type, tail_index, k_values=[1, 3, 10, 50, 100], batch_size=4096, device='cpu'):
    model.eval() # Set model to evaluation mode

    ranks = []
    hits_at_k = {k: 0 for k in k_values}
    num_samples = head_index.size(0)

    # Create a data loader for evaluation to process in batches
    eval_loader = model.transe.loader(
        head_index=head_index,
        rel_type=rel_type,
        tail_index=tail_index,
        batch_size=batch_size,
        shuffle=False,
    )

    with torch.no_grad(): # Disable gradient calculation for inference
        for head, rel, tail in tqdm(eval_loader, desc="Evaluating Link Prediction"):
            head, rel, tail = head.to(device), rel.to(device), tail.to(device)

            # For each (head, relation, tail) triple, we'll evaluate two tasks:
            # 1. Predicting the tail entity given (head, relation)
            # 2. Predicting the head entity given (relation, tail)

            # --- Tail Prediction --- (corrupting tail)
            # Generate corrupted triples: (head, relation, all_possible_tails)
            all_nodes = torch.arange(model.num_nodes, device=device)
            # Expand head and rel for broadcasting against all_nodes
            b_head = head.unsqueeze(1).expand(-1, model.num_nodes)
            b_rel = rel.unsqueeze(1).expand(-1, model.num_nodes)
            b_tail_corrupted = all_nodes.unsqueeze(0).expand(head.size(0), -1)

            # Calculate scores for all corrupted triples
            # The score function is defined by the TransE model itself (distance)
            # Lower distance is better, so we sort in ascending order
            scores_tail_pred = model.transe.forward(b_head, b_rel, b_tail_corrupted).squeeze()

            # Get the score of the true tail for each batch item
            true_tail_scores = scores_tail_pred.gather(1, tail.unsqueeze(1)).squeeze()

            # Calculate rank: count how many corrupted tails have a better (lower) score than the true tail
            # Add 1 to convert from 0-indexed count to 1-indexed rank
            ranks_tail_pred = (scores_tail_pred < true_tail_scores.unsqueeze(1)).sum(dim=1) + 1
            ranks.extend(ranks_tail_pred.cpu().tolist())

            # Calculate Hits@k for tail prediction
            for k in k_values:
                hits_at_k[k] += (ranks_tail_pred <= k).sum().item()

            # --- Head Prediction --- (corrupting head)
            # Generate corrupted triples: (all_possible_heads, relation, tail)
            b_head_corrupted = all_nodes.unsqueeze(0).expand(head.size(0), -1)
            b_rel = rel.unsqueeze(1).expand(-1, model.num_nodes)
            b_tail = tail.unsqueeze(1).expand(-1, model.num_nodes)

            # Calculate scores for all corrupted triples
            scores_head_pred = model.transe.forward(b_head_corrupted, b_rel, b_tail).squeeze()

            # Get the score of the true head for each batch item
            true_head_scores = scores_head_pred.gather(1, head.unsqueeze(1)).squeeze()

            # Calculate rank
            ranks_head_pred = (scores_head_pred < true_head_scores.unsqueeze(1)).sum(dim=1) + 1
            ranks.extend(ranks_head_pred.cpu().tolist())

            # Calculate Hits@k for head prediction
            for k in k_values:
                hits_at_k[k] += (ranks_head_pred <= k).sum().item()

    # Aggregate results
    mean_rank = sum(ranks) / len(ranks)
    for k in k_values:
        hits_at_k[k] /= (num_samples * 2) # Multiply by 2 because we did head and tail prediction for each sample

    results = {"mean_rank": mean_rank}
    results.update({f"hits_at_{k}": hits_at_k[k] for k in k_values})

    return results

print("Defined evaluate_link_prediction function.")

## Call Link Prediction Evaluation

### Subtask:
Integrate the newly created link prediction evaluation function into the main script to calculate and print the Hits@10 metric.


**Reasoning**:
The subtask requires calling the newly defined `evaluate_link_prediction` function and printing its results. This code block will execute the function on the test set and display the calculated Hits@k and Mean Rank.



In [ ]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np # For random sampling

@torch.no_grad()
def evaluate_link_prediction(transe_model, edge_index, edge_type, eval_indices, k_values=[1, 3, 10], num_negatives=100, batch_size=4096):
    """
    Evaluates link prediction performance using Hits@k and Mean Rank by manually
    corrupting entities with negative sampling and ranking scores.

    Args:
        transe_model: The TransE model (e.g., model.transe) with trained embeddings.
        edge_index: Tensor of shape [2, num_edges] representing head and tail indices.
        edge_type: Tensor of shape [num_edges] representing relation types.
        eval_indices: Indices for the evaluation set (e.g., test_idx).
        k_values: List of k values for Hits@k.
        num_negatives: Number of negative samples to generate for each positive triple.
        batch_size: Batch size for processing evaluation triples.
    Returns:
        A dictionary containing Hits@k and Mean Rank metrics.
    """
    transe_model.eval()
    device = transe_model.node_emb.weight.device # Get the device from the model's embeddings

    # Extract head, relation, tail for the evaluation triples
    head_indices = edge_index[0, eval_indices]
    rel_types = edge_type[eval_indices]
    tail_indices = edge_index[1, eval_indices]

    ranks = []
    hits_at_k = {k: 0 for k in k_values}

    num_eval_triples = head_indices.size(0)

    # DataLoader for evaluation triples
    eval_dataset = torch.utils.data.TensorDataset(head_indices, rel_types, tail_indices)
    eval_loader = torch.utils.data.DataLoader(eval_dataset, batch_size=batch_size, shuffle=False)

    print(f"\nPerforming link prediction evaluation on {num_eval_triples} triples with {num_negatives} negative samples per triple...")

    for h_batch, r_batch, t_batch in tqdm(eval_loader, desc="Evaluating Link Prediction"):
        h_batch, r_batch, t_batch = h_batch.to(device), r_batch.to(device), t_batch.to(device)

        batch_size_actual = h_batch.size(0)
        num_nodes = transe_model.num_nodes

        # --- Tail Prediction ---
        # Generate negative tails for each triple in the batch
        # Ensure negative tails are not the true tail for each corresponding batch item
        neg_tails = torch.randint(0, num_nodes, (batch_size_actual, num_negatives), device=device)
        # For simplicity, if a negative happens to be the true tail, we keep it for now.
        # A more rigorous implementation would re-sample until all are distinct from the true tail.

        # Concatenate true tails and negative tails for ranking
        # candidate_tails: [batch_size, 1 + num_negatives]
        candidate_tails = torch.cat([t_batch.unsqueeze(1), neg_tails], dim=1)

        # Expand h_batch and r_batch to match the number of candidates
        # h_expanded: [batch_size, 1 + num_negatives]
        # r_expanded: [batch_size, 1 + num_negatives]
        h_expanded = h_batch.unsqueeze(1).expand(-1, 1 + num_negatives)
        r_expanded = r_batch.unsqueeze(1).expand(-1, 1 + num_negatives)

        # Score all candidate triples (h, r, candidate_t)
        # scores: [batch_size, 1 + num_negatives]
        scores = transe_model.forward(
            h_expanded.reshape(-1),
            r_expanded.reshape(-1),
            candidate_tails.reshape(-1)
        ).reshape(batch_size_actual, 1 + num_negatives)

        # Rank the true tail (which is at index 0 in candidate_tails)
        # Scores are distances, so lower is better. Sort in ascending order.
        sorted_indices = torch.argsort(scores, dim=1) # indices of sorted scores

        # Find rank of the true tail (which was the first candidate)
        # The true rank is the position of index 0 in sorted_indices, plus 1 (1-based rank)
        true_tail_rank = (sorted_indices == 0).nonzero(as_tuple=True)[1] + 1
        ranks.extend(true_tail_rank.cpu().tolist()) # Collect ranks for Mean Rank

        # Update Hits@k
        for k in k_values:
            hits_at_k[k] += (true_tail_rank <= k).sum().item()


        # --- Head Prediction --- (symmetric logic)
        # Generate negative heads for each triple in the batch
        neg_heads = torch.randint(0, num_nodes, (batch_size_actual, num_negatives), device=device)

        # Concatenate true heads and negative heads for ranking
        candidate_heads = torch.cat([h_batch.unsqueeze(1), neg_heads], dim=1)

        # Expand r_batch and t_batch to match the number of candidates
        r_expanded = r_batch.unsqueeze(1).expand(-1, 1 + num_negatives)
        t_expanded = t_batch.unsqueeze(1).expand(-1, 1 + num_negatives)

        # Score all candidate triples (candidate_h, r, t)
        scores = transe_model.forward(
            candidate_heads.reshape(-1),
            r_expanded.reshape(-1),
            t_expanded.reshape(-1)
        ).reshape(batch_size_actual, 1 + num_negatives)

        # Rank the true head (which is at index 0 in candidate_heads)
        sorted_indices = torch.argsort(scores, dim=1)
        true_head_rank = (sorted_indices == 0).nonzero(as_tuple=True)[1] + 1
        ranks.extend(true_head_rank.cpu().tolist()) # Collect ranks for Mean Rank

        # Update Hits@k
        for k in k_values:
            hits_at_k[k] += (true_head_rank <= k).sum().item()

    # Aggregate results
    mean_rank = sum(ranks) / len(ranks) if len(ranks) > 0 else 0.0
    total_evaluated_predictions = num_eval_triples * 2 # Each original sample leads to head and tail prediction

    final_hits = {}
    for k in k_values:
        final_hits[f"Hits@{k}"] = hits_at_k[k] / total_evaluated_predictions

    results = {"Mean Rank": mean_rank}
    results.update(final_hits)

    return results

print("\n--- Evaluating Link Prediction on Test Set ---")
link_pred_metrics = evaluate_link_prediction(
    transe_model=model.transe, # Pass model.transe directly as it's the TransE instance
    edge_index=graph_data["edge_index"],
    edge_type=graph_data["edge_type"],
    eval_indices=graph_data["test_idx"], # Evaluate on the test set
    k_values=[1, 3, 10],
    num_negatives=100, # Use num_negatives for sampling
    batch_size=BATCH_SIZE # Ensure BATCH_SIZE is accessible (it is in kernel state)
)

print("-------------------------------------------------------")
print("Final Link Prediction Metrics (Test Set):")
for metric, value in link_pred_metrics.items():
    print(f"  {metric}: {value:.4f}")
print("-------------------------------------------------------")

# Task
**Define AdaptedComplEx Model**: Create a new Python class `AdaptedComplEx` that inherits from `nn.Module`. This class will encapsulate `torch_geometric.nn.ComplEx` for embedding learning and add a multi-head classifier for relation classification. It will follow the same pattern as `AdaptedTransE`, ensuring the `forward` method produces logits for relation classification and a `complex_loss` method returns the ComplEx specific loss for pre-training embeddings.

In [ ]:
# Install necessary libraries if not already present
!pip install torch_geometric

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import ComplEx


class AdaptedComplEx(nn.Module):
    def __init__(self, num_nodes, num_relations, embedding_dim, dropout_rate=0.3):
        super().__init__()
        self.num_nodes = num_nodes
        self.num_relations = num_relations
        self.embedding_dim = embedding_dim

        # 1. Embedding Layer (ComplEx from PyG)
        # ComplEx uses hidden_channels for the dimension of *each* part (real/imaginary).
        # So, total embedding dimension for each node will be 2 * embedding_dim.
        self.complex = ComplEx(
            num_nodes=num_nodes,
            num_relations=num_relations,
            hidden_channels=embedding_dim, # This is the dimension for the real and imaginary parts separately
            margin=1.0,
        )

        # 2. Relation Classifier (Multi-Head Classifier)
        # It takes the difference vector (h - t), where h and t are the concatenated
        # real and imaginary parts of the embeddings. Thus, input dimension is 2 * embedding_dim.
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout_rate),
            nn.Linear(2 * embedding_dim, num_relations) # ComplEx uses 2*embedding_dim for its node embeddings
        )

        # Initialize the classifier's weights
        nn.init.xavier_uniform_(self.classifier[1].weight)

    def forward(self, head_index, tail_index):
        # 1. Get Entity Embeddings from the ComplEx layer
        # ComplEx's node_emb returns embeddings that are already concatenated
        # real and imaginary parts, effectively a 2*embedding_dim vector.
        h = self.complex.node_emb(head_index)
        t = self.complex.node_emb(tail_index)

        # 2. Calculate the difference vector (h - t)
        diff_vector = h - t

        # 3. Pass the difference vector to the classifier head
        logits = self.classifier(diff_vector)

        return logits

    # We will use the standard PyG ComplEx loss for pre-training the embeddings
    def complex_loss(self, head_index, rel_type, tail_index):
        return self.complex.loss(head_index, rel_type, tail_index)

print("AdaptedComplEx class defined successfully.")

AdaptedComplEx class defined successfully.


## Train AdaptedComplEx Model

### Subtask:
Implement the training procedure for the `AdaptedComplEx` model, including pre-training of ComplEx embeddings and training of the classifier head.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.nn import ComplEx
from tqdm import tqdm
import os

# --- 1. Custom Classifier Module (Copied for completeness and independent execution)
#      This was defined in the previous step, ensuring it's available.
class AdaptedComplEx(nn.Module):
    def __init__(self, num_nodes, num_relations, embedding_dim, dropout_rate=0.3):
        super().__init__()
        self.num_nodes = num_nodes
        self.num_relations = num_relations
        self.embedding_dim = embedding_dim

        self.complex = ComplEx(
            num_nodes=num_nodes,
            num_relations=num_relations,
            hidden_channels=embedding_dim,
            # Removed 'margin=1.0' as ComplEx does not support it
        )

        # Based on the error, the ComplEx node embeddings are effectively `embedding_dim`,
        # not `2 * embedding_dim` as initially assumed for the input to the classifier.
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout_rate),
            nn.Linear(embedding_dim, num_relations) # Changed from 2 * embedding_dim
        )

        nn.init.xavier_uniform_(self.classifier[1].weight)

    def forward(self, head_index, tail_index):
        h = self.complex.node_emb(head_index)
        t = self.complex.node_emb(tail_index)

        diff_vector = h - t

        logits = self.classifier(diff_vector)

        return logits

    def complex_loss(self, head_index, rel_type, tail_index):
        return self.complex.loss(head_index, rel_type, tail_index)


# --- 2. Training Function for the AdaptedComplEx Classifier ---
def train_complex_classifier(model, optimizer, edge_index, edge_type, base_relations, num_epochs, batch_size, device):
    model.train()
    num_base_relations = len(base_relations)

    # Filter for forward (non-inverse) relations in the training set
    forward_mask = edge_type < num_base_relations
    train_indices_forward = graph_data['train_idx'][torch.isin(graph_data['train_idx'], torch.where(forward_mask)[0])]

    # Create a new loader that iterates over the forward-only triples in the training set
    classifier_loader = model.complex.loader(
        head_index=edge_index[0, train_indices_forward],
        rel_type=edge_type[train_indices_forward],
        tail_index=edge_index[1, train_indices_forward],
        batch_size=batch_size,
        shuffle=True,
    )

    print("\nStarting Adapted ComplEx Classifier Training...")

    for epoch in range(1, num_epochs + 1):
        total_loss = 0
        correct_predictions = 0
        total_samples = 0

        for head_index, rel_type_batch, tail_index in classifier_loader:
            head_index, rel_type_batch, tail_index = head_index.to(device), rel_type_batch.to(device), tail_index.to(device)
            optimizer.zero_grad()

            logits = model(head_index, tail_index)
            loss = F.cross_entropy(logits, rel_type_batch)

            loss.backward()
            optimizer.step()

            total_loss += loss.item() * head_index.size(0)
            total_samples += head_index.size(0)

            _, predicted = torch.max(logits, 1)
            correct_predictions += (predicted == rel_type_batch).sum().item()

        avg_loss = total_loss / total_samples
        accuracy = correct_predictions / total_samples

        print(f"Epoch: {epoch:03d}, Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.4f}")

        if epoch % 5 == 0 or epoch == num_epochs:
            val_accuracy = evaluate_complex_classifier(model, edge_index, edge_type, graph_data['val_idx'], num_base_relations, device)
            print(f"  >>> Validation Accuracy: {val_accuracy:.4f}")

# --- 3. Evaluation Function for the AdaptedComplEx Classifier ---
@torch.no_grad()
def evaluate_complex_classifier(model, edge_index, edge_type, eval_idx, num_base_relations, device):
    model.eval()

    # Select only the forward (non-inverse) relations in the evaluation set
    eval_mask = edge_type < num_base_relations
    eval_indices_forward = eval_idx[torch.isin(eval_idx, torch.where(eval_mask)[0])]

    if eval_indices_forward.numel() == 0:
        return 0.0

    head_index = edge_index[0, eval_indices_forward]
    tail_index = edge_index[1, eval_indices_forward]
    rel_type = edge_type[eval_indices_forward]

    logits = model(head_index, tail_index)
    _, predicted = torch.max(logits, 1)

    correct_predictions = (predicted == rel_type).sum().item()
    total_samples = rel_type.size(0)

    return correct_predictions / total_samples


# --- 4. Main Execution (ComplEx Pre-training + Classifier Training) ---
# Configuration (ensure these are loaded from previous steps or defined)
EMBEDDING_DIM = 200      # Example value
CLASSIFIER_LR = 0.001    # Example value
CLASSIFIER_EPOCHS = 10   # Example value
PRE_TRAIN_EPOCHS = 50    # Example value
BATCH_SIZE = 4096         # Example value
GRAPH_DATA_PATH = "graph_data.pt" # Example path

if not os.path.exists(GRAPH_DATA_PATH):
    raise FileNotFoundError(f"{GRAPH_DATA_PATH} not found. Please ensure data preparation is complete.")

graph_data = torch.load(GRAPH_DATA_PATH)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

num_nodes = graph_data["num_nodes"]
num_relations = graph_data["num_relations"]
base_relations = graph_data["base_relations"]

# Move all relevant tensors to the device
graph_data["edge_index"] = graph_data["edge_index"].to(device)
graph_data["edge_type"] = graph_data["edge_type"].to(device)
graph_data["train_idx"] = graph_data["train_idx"].to(device)
graph_data["val_idx"] = graph_data["val_idx"].to(device)
graph_data["test_idx"] = graph_data["test_idx"].to(device)

edge_index = graph_data["edge_index"]
edg_type = graph_data["edge_type"]

# Initialize AdaptedComplEx Model
model_complex = AdaptedComplEx(num_nodes, num_relations, EMBEDDING_DIM).to(device)

# --- PHASE 1: ComplEx Embedding Pre-training ---
print("\n--- PHASE 1: ComplEx Embedding Pre-training ---")
pre_train_optimizer_complex = optim.Adam(model_complex.complex.parameters(), lr=0.01)
complex_loader = model_complex.complex.loader(
    head_index=edge_index[0],
    rel_type=edg_type,
    tail_index=edge_index[1],
    batch_size=BATCH_SIZE,
    shuffle=True,
)

for epoch in tqdm(range(1, PRE_TRAIN_EPOCHS + 1), desc="ComplEx Pre-training"):
    total_loss = 0
    for head_index, rel_type_batch, tail_index in complex_loader:
        pre_train_optimizer_complex.zero_grad()
        loss = model_complex.complex_loss(head_index.to(device), rel_type_batch.to(device), tail_index.to(device))
        loss.backward()
        pre_train_optimizer_complex.step()
        total_loss += float(loss) * head_index.numel()

    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d} Loss: {total_loss / edge_index.size(1):.4f}")

# --- PHASE 2: Relation Classification Training ---
print("\n--- PHASE 2: Relation Classification Training (AdaptedComplEx) ---")

# Freeze ComplEx embeddings
for param in model_complex.complex.parameters():
    param.requires_grad = False

classifier_optimizer_complex = optim.Adam(model_complex.classifier.parameters(), lr=CLASSIFIER_LR)

train_complex_classifier(model_complex, classifier_optimizer_complex, edge_index, edg_type, base_relations, CLASSIFIER_EPOCHS, BATCH_SIZE, device)

# Final Test Accuracy
test_accuracy_complex = evaluate_complex_classifier(model_complex, edge_index, edg_type, graph_data['test_idx'], len(base_relations), device)
print("\n--- FINAL CLASSIFIER TEST RESULTS (AdaptedComplEx) ---")
print(f"Test Accuracy: {test_accuracy_complex:.4f}")
print("----------------------------------------------------")

Using device: cuda

--- PHASE 1: ComplEx Embedding Pre-training ---


ComplEx Pre-training:  20%|██        | 10/50 [01:50<07:20, 11.01s/it]

Epoch 010 Loss: 0.0217


ComplEx Pre-training:  40%|████      | 20/50 [03:40<05:30, 11.01s/it]

Epoch 020 Loss: 0.0147


...

ComplEx Pre-training:  80%|████████  | 40/50 [07:20<01:50, 11.03s/it]

Epoch 040 Loss: 0.0123


ComplEx Pre-training: 100%|██████████| 50/50 [09:10<00:00, 11.02s/it]

Epoch 050 Loss: 0.0127

--- PHASE 2: Relation Classification Training (AdaptedComplEx) ---

Starting Adapted ComplEx Classifier Training...


Epoch: 001, Loss: 4.2449, Train Accuracy: 0.2234
Epoch: 002, Loss: 3.2637, Train Accuracy: 0.3767
Epoch: 003, Loss: 3.0139, Train Accuracy: 0.4046
Epoch: 004, Loss: 2.8115, Train Accuracy: 0.4208
Epoch: 005, Loss: 2.6202, Train Accuracy: 0.4382
  >>> Validation Accuracy: 0.5266
Epoch: 006, Loss: 2.1802, Train Accuracy: 0.5292
Epoch: 007, Loss: 2.0035, Train Accuracy: 0.5527
Epoch: 008, Loss: 1.8443, Train Accuracy: 0.5841
Epoch: 009, Loss: 1.6923, Train Accuracy: 0.6210
Epoch: 010, Loss: 1.5497, Train Accuracy: 0.6612
  >>> Validation Accuracy: 0.6818

--- FINAL CLASSIFIER TEST RESULTS (AdaptedComplEx) ---
Test Accuracy: 0.6789
----------------------------------------------------


# Task
Final Task: Evaluate the link prediction performance of the `AdaptedComplEx` model on the test set using Hits@k and Mean Rank metrics, and summarize the results.

## Define Link Prediction Evaluation Function for ComplEx

### Subtask:
Create a Python function `evaluate_link_prediction_complex` that takes the trained `ComplEx` embedding model, `edge_index`, `edge_type`, `eval_indices`, `k_values`, `num_negatives`, and `batch_size` as input to evaluate link prediction.


**Reasoning**:
The subtask requires defining a new function `evaluate_link_prediction_complex` that uses the `ComplEx` model's `k_hit` method to calculate link prediction metrics. This code block will implement that function.



In [ ]:
import torch
import torch.nn.functional as F
from tqdm import tqdm

@torch.no_grad()
def evaluate_link_prediction_complex(complex_model, edge_index, edge_type, eval_indices, k_values=[1, 3, 10], num_negatives=100, batch_size=4096):
    """
    Evaluates link prediction performance using Hits@k and Mean Rank for the ComplEx model.

    Args:
        complex_model: The ComplEx model instance (e.g., model_complex.complex).
        edge_index: Tensor of shape [2, num_edges] representing head and tail indices.
        edge_type: Tensor of shape [num_edges] representing relation types.
        eval_indices: Indices for the evaluation set (e.g., test_idx).
        k_values: List of k values for Hits@k.
        num_negatives: Number of negative samples to generate for each positive triple.
        batch_size: Batch size for processing evaluation triples.
    Returns:
        A dictionary containing Hits@k and Mean Rank metrics.
    """
    complex_model.eval() # Set model to evaluation mode

    # Extract head, relation, tail for the evaluation triples
    head_indices = edge_index[0, eval_indices]
    rel_types = edge_type[eval_indices]
    tail_indices = edge_index[1, eval_indices]

    results = {}
    print(f"\nPerforming link prediction evaluation on {len(eval_indices)} triples with {num_negatives} negative samples per triple using ComplEx...")

    # PyG's k_hit function internally computes mean rank and Hits@k by corrupting
    # both head and tail entities and averaging the results.
    hits_at_k, mean_rank = complex_model.k_hit(
        head_index=head_indices,
        rel_type=rel_types,
        tail_index=tail_indices,
        k=k_values[-1], # k_hit expects a single k, it internally calculates for all k up to this value
        num_negatives=num_negatives,
        batch_size=batch_size
    )

    # The k_hit function returns Hits@k for all specified k values in an internal list,
    # and also the mean rank.
    # We need to map the output to our k_values.
    # The `k_hit` function returns a tensor of Hits@k values (indexed by k-1 for each k in k_values) and the mean rank.
    # Let's assume hits_at_k is a tensor where hits_at_k[i] corresponds to Hits@k_values[i]

    # For PyG's ComplEx k_hit, it returns one Hits@k value (for the k specified) and MR
    # If k_hit changes behavior, this needs to be adjusted. For now, we will interpret the single k_hit output
    # as the maximum k in k_values.

    # Re-calling k_hit for each k for clarity based on typical PyG behavior
    # A more efficient implementation might use a single call if PyG's k_hit returns all at once.
    for k_val in k_values:
        current_hits_at_k, current_mean_rank = complex_model.k_hit(
            head_index=head_indices,
            rel_type=rel_types,
            tail_index=tail_indices,
            k=k_val,
            num_negatives=num_negatives,
            batch_size=batch_size
        )
        results[f"Hits@{k_val}"] = current_hits_at_k.item() # .item() to get scalar from tensor
        print(f"  Hits@{k_val}: {results[f'Hits@{k_val}']:.4f}")

    # Mean Rank is computed with each k_hit call, but usually, we report one overall MR.
    # We'll use the last computed mean rank for simplicity, or re-run k_hit for an overall MR if needed.
    # A better approach would be to collect all ranks and compute MR once.
    # For now, let's use the one from the largest k or if k_hit returns a single MR.
    # Based on torch_geometric.nn.kge.k_hit, it returns one hits_at_k and one mean_rank.
    # So we'll run it once for a representative k (e.g., k=10) to get the MR.

    # Let's ensure we get one consistent Mean Rank
    _, final_mean_rank = complex_model.k_hit(
        head_index=head_indices,
        rel_type=rel_types,
        tail_index=tail_indices,
        k=k_values[-1], # Use the largest k to get the mean rank
        num_negatives=num_negatives,
        batch_size=batch_size
    )
    results["Mean Rank"] = final_mean_rank.item()
    print(f"  Mean Rank: {results['Mean Rank']:.4f}")

    return results

print("Defined evaluate_link_prediction_complex function.")

Defined evaluate_link_prediction_complex function.


In [ ]:
print("\n--- Evaluating Link Prediction on Test Set (AdaptedComplEx) ---")
link_pred_metrics_complex = evaluate_link_prediction_complex(
    complex_model=model_complex.complex, # Pass model_complex.complex directly
    edge_index=graph_data["edge_index"],
    edge_type=graph_data["edge_type"],
    eval_indices=graph_data["test_idx"], # Evaluate on the test set
    k_values=[1, 3, 10],
    num_negatives=100, # Use num_negatives for sampling
    batch_size=BATCH_SIZE # Ensure BATCH_SIZE is accessible
)

print("-------------------------------------------------------")
print("Final Link Prediction Metrics (Test Set for ComplEx):")
for metric, value in link_pred_metrics_complex.items():
    print(f"  {metric}: {value:.4f}")
print("-------------------------------------------------------")


--- Evaluating Link Prediction on Test Set (AdaptedComplEx) ---

Performing link prediction evaluation on 259860 triples with 100 negative samples per triple using ComplEx...


AttributeError: 'ComplEx' object has no attribute 'k_hit'

**Reasoning**:
The error `AttributeError: 'ComplEx' object has no attribute 'k_hit'` indicates that the `torch_geometric.nn.ComplEx` class does not have a built-in `k_hit` method. I need to modify the `evaluate_link_prediction_complex` function to manually perform negative sampling, score calculation, and rank aggregation, similar to how the `evaluate_link_prediction` function for TransE was implemented.



In [ ]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np # Needed for random sampling

@torch.no_grad()
def evaluate_link_prediction_complex(complex_model, edge_index, edge_type, eval_indices, k_values=[1, 3, 10], num_negatives=100, batch_size=4096):
    """
    Evaluates link prediction performance using Hits@k and Mean Rank for the ComplEx model
    by manually corrupting entities with negative sampling and ranking scores.

    Args:
        complex_model: The ComplEx model instance (e.g., model_complex.complex).
        edge_index: Tensor of shape [2, num_edges] representing head and tail indices.
        edge_type: Tensor of shape [num_edges] representing relation types.
        eval_indices: Indices for the evaluation set (e.g., test_idx).
        k_values: List of k values for Hits@k.
        num_negatives: Number of negative samples to generate for each positive triple.
        batch_size: Batch size for processing evaluation triples.
    Returns:
        A dictionary containing Hits@k and Mean Rank metrics.
    """
    complex_model.eval() # Set model to evaluation mode
    device = complex_model.node_emb.weight.device # Get the device from the model's embeddings

    # Extract head, relation, tail for the evaluation triples
    head_indices = edge_index[0, eval_indices]
    rel_types = edge_type[eval_indices]
    tail_indices = edge_index[1, eval_indices]

    ranks = []
    hits_at_k = {k: 0 for k in k_values}

    num_eval_triples = head_indices.size(0)
    num_nodes = complex_model.num_nodes # Get total number of nodes from the model

    # DataLoader for evaluation triples
    eval_dataset = torch.utils.data.TensorDataset(head_indices, rel_types, tail_indices)
    eval_loader = torch.utils.data.DataLoader(eval_dataset, batch_size=batch_size, shuffle=False)

    print(f"\nPerforming link prediction evaluation on {num_eval_triples} triples with {num_negatives} negative samples per triple using ComplEx...")

    for h_batch, r_batch, t_batch in tqdm(eval_loader, desc="Evaluating Link Prediction"):
        h_batch, r_batch, t_batch = h_batch.to(device), r_batch.to(device), t_batch.to(device)

        batch_size_actual = h_batch.size(0)

        # --- Tail Prediction ---
        # Generate negative tails for each triple in the batch
        # For simplicity, negative samples might include the true tail, which is handled by ranking.
        neg_tails = torch.randint(0, num_nodes, (batch_size_actual, num_negatives), device=device)

        # Concatenate true tails and negative tails for ranking
        # candidate_tails: [batch_size, 1 + num_negatives]
        candidate_tails = torch.cat([t_batch.unsqueeze(1), neg_tails], dim=1)

        # Expand h_batch and r_batch to match the number of candidates
        h_expanded = h_batch.unsqueeze(1).expand(-1, 1 + num_negatives)
        r_expanded = r_batch.unsqueeze(1).expand(-1, 1 + num_negatives)

        # Score all candidate triples (h, r, candidate_t)
        # scores: [batch_size, 1 + num_negatives]
        scores = complex_model( # Use the forward method of ComplEx model for scoring
            h_expanded.reshape(-1),
            r_expanded.reshape(-1),
            candidate_tails.reshape(-1)
        ).reshape(batch_size_actual, 1 + num_negatives)

        # Rank the true tail (which is at index 0 in candidate_tails)
        # Scores are negative probabilities/likelihoods (ComplEx aims to maximize scores), so higher is better. Sort in descending order.
        sorted_indices = torch.argsort(scores, dim=1, descending=True) # indices of sorted scores

        # Find rank of the true tail (which was the first candidate)
        true_tail_rank = (sorted_indices == 0).nonzero(as_tuple=True)[1] + 1
        ranks.extend(true_tail_rank.cpu().tolist()) # Collect ranks for Mean Rank

        # Update Hits@k
        for k in k_values:
            hits_at_k[k] += (true_tail_rank <= k).sum().item()


        # --- Head Prediction --- (symmetric logic)
        # Generate negative heads for each triple in the batch
        neg_heads = torch.randint(0, num_nodes, (batch_size_actual, num_negatives), device=device)

        # Concatenate true heads and negative heads for ranking
        candidate_heads = torch.cat([h_batch.unsqueeze(1), neg_heads], dim=1)

        # Expand r_batch and t_batch to match the number of candidates
        r_expanded = r_batch.unsqueeze(1).expand(-1, 1 + num_negatives)
        t_expanded = t_batch.unsqueeze(1).expand(-1, 1 + num_negatives)

        # Score all candidate triples (candidate_h, r, t)
        scores = complex_model( # Use the forward method of ComplEx model for scoring
            candidate_heads.reshape(-1),
            r_expanded.reshape(-1),
            t_expanded.reshape(-1)
        ).reshape(batch_size_actual, 1 + num_negatives)

        # Rank the true head (which is at index 0 in candidate_heads)
        sorted_indices = torch.argsort(scores, dim=1, descending=True)
        true_head_rank = (sorted_indices == 0).nonzero(as_tuple=True)[1] + 1
        ranks.extend(true_head_rank.cpu().tolist()) # Collect ranks for Mean Rank

        # Update Hits@k
        for k in k_values:
            hits_at_k[k] += (true_head_rank <= k).sum().item()

    # Aggregate results
    mean_rank = sum(ranks) / len(ranks) if len(ranks) > 0 else 0.0
    total_evaluated_predictions = num_eval_triples * 2 # Each original sample leads to head and tail prediction

    final_hits = {}
    for k in k_values:
        final_hits[f"Hits@{k}"] = hits_at_k[k] / total_evaluated_predictions

    results = {"Mean Rank": mean_rank}
    results.update(final_hits)

    return results

print("Redefined evaluate_link_prediction_complex function with manual ranking.")

Redefined evaluate_link_prediction_complex function with manual ranking.


**Reasoning**:
Now that the `evaluate_link_prediction_complex` function has been redefined to correctly handle link prediction evaluation for ComplEx, I will call this function to compute the Hits@k and Mean Rank metrics on the test set and print the summary results.



In [ ]:
print("\n--- Evaluating Link Prediction on Test Set (AdaptedComplEx) ---")
link_pred_metrics_complex = evaluate_link_prediction_complex(
    complex_model=model_complex.complex, # Pass model_complex.complex directly
    edge_index=graph_data["edge_index"],
    edge_type=graph_data["edge_type"],
    eval_indices=graph_data["test_idx"], # Evaluate on the test set
    k_values=[1, 3, 10],
    num_negatives=100, # Use num_negatives for sampling
    batch_size=BATCH_SIZE # Ensure BATCH_SIZE is accessible
)

print("-------------------------------------------------------")
print("Final Link Prediction Metrics (Test Set for ComplEx):")
for metric, value in link_pred_metrics_complex.items():
    print(f"  {metric}: {value:.4f}")
print("-------------------------------------------------------")


--- Evaluating Link Prediction on Test Set (AdaptedComplEx) ---

Performing link prediction evaluation on 259860 triples with 100 negative samples per triple using ComplEx...


Evaluating Link Prediction: 100%|██████████| 64/64 [00:05<00:00, 10.96it/s]

-------------------------------------------------------
Final Link Prediction Metrics (Test Set for ComplEx):
  Mean Rank: 1.0714
  Hits@1: 0.9624
  Hits@3: 0.9928
  Hits@10: 0.9999
-------------------------------------------------------


# Task
Revise the `evaluate_link_prediction_complex` function to perform link prediction evaluation against all possible entities in the graph by scoring the true triple and all corrupted triples (where either the head or tail is replaced by every other entity in the graph) to determine the rank of the true entity. Then, re-evaluate the `AdaptedComplEx` model's link prediction performance using this revised function and display the new Hits@k and Mean Rank metrics.

## Revise Link Prediction Evaluation Function

### Subtask:
Modify the `evaluate_link_prediction_complex` function to perform link prediction evaluation against *all* possible entities in the graph (i.e., all `num_nodes` entities).


**Reasoning**:
The subtask requires modifying the link prediction evaluation function to test against all possible entities. I will define a new `evaluate_link_prediction_complex` function that iterates through true triples, generates all `num_nodes` candidates in chunks, scores them using `complex_model.forward`, and then calculates ranks and Hits@k metrics based on these full rankings.



In [ ]:
import torch
import torch.nn.functional as F
from tqdm import tqdm

@torch.no_grad()
def evaluate_link_prediction_complex(complex_model, edge_index, edge_type, eval_indices, k_values=[1, 3, 10], batch_size_true_triples=1, max_candidate_batch_size=1024):
    """
    Evaluates link prediction performance for the ComplEx model against all possible entities
    in the graph, by processing candidate entities in chunks to manage memory.

    Args:
        complex_model: The ComplEx model instance (e.g., model_complex.complex).
        edge_index: Tensor of shape [2, num_edges] representing head and tail indices.
        edge_type: Tensor of shape [num_edges] representing relation types.
        eval_indices: Indices for the evaluation set (e.g., test_idx).
        k_values: List of k values for Hits@k.
        batch_size_true_triples: Number of true triples to process at once. Recommended 1.
        max_candidate_batch_size: Max number of candidate entities to score at once to manage memory.
    Returns:
        A dictionary containing Hits@k and Mean Rank metrics.
    """
    complex_model.eval() # Set model to evaluation mode
    device = complex_model.node_emb.weight.device # Get the device from the model's embeddings

    # Extract head, relation, tail for the evaluation triples
    head_indices = edge_index[0, eval_indices]
    rel_types = edge_type[eval_indices]
    tail_indices = edge_index[1, eval_indices]

    ranks = []
    hits_at_k = {k: 0 for k in k_values}

    num_eval_triples = head_indices.size(0)
    num_nodes = complex_model.num_nodes # Get total number of nodes from the model
    all_nodes = torch.arange(num_nodes, device=device)

    # DataLoader for evaluation triples, processing one true triple at a time
    eval_dataset = torch.utils.data.TensorDataset(head_indices, rel_types, tail_indices)
    eval_loader = torch.utils.data.DataLoader(eval_dataset, batch_size=batch_size_true_triples, shuffle=False)

    print(f"\nPerforming link prediction evaluation on {num_eval_triples} triples (against all {num_nodes} entities per triple) using ComplEx...")

    for h_true_batch, r_true_batch, t_true_batch in tqdm(eval_loader, desc="Evaluating Link Prediction (all candidates)"):
        h_true = h_true_batch.squeeze(0) # Get single true head
        r_true = r_true_batch.squeeze(0) # Get single true relation
        t_true = t_true_batch.squeeze(0) # Get single true tail

        # --- Tail Prediction: Evaluate (h_true, r_true, ?) ---
        all_tail_scores = []
        for i in range(0, num_nodes, max_candidate_batch_size):
            candidate_tails_chunk = all_nodes[i : i + max_candidate_batch_size]

            # Expand true head and relation for the chunk of candidate tails
            h_expanded = h_true.expand_as(candidate_tails_chunk)
            r_expanded = r_true.expand_as(candidate_tails_chunk)

            # Score the chunk of candidate triples (h_true, r_true, candidate_tail)
            scores_chunk = complex_model(h_expanded, r_expanded, candidate_tails_chunk)
            all_tail_scores.append(scores_chunk)

        # Concatenate scores for all candidates and find rank of the true tail
        scores_all_tails = torch.cat(all_tail_scores)
        # ComplEx maximizes scores, so rank is determined by how many candidates have a higher score
        rank_t = (scores_all_tails > scores_all_tails[t_true]).sum().item() + 1
        ranks.append(rank_t)
        for k in k_values:
            if rank_t <= k:
                hits_at_k[k] += 1

        # --- Head Prediction: Evaluate (?, r_true, t_true) ---
        all_head_scores = []
        for i in range(0, num_nodes, max_candidate_batch_size):
            candidate_heads_chunk = all_nodes[i : i + max_candidate_batch_size]

            # Expand true relation and tail for the chunk of candidate heads
            r_expanded = r_true.expand_as(candidate_heads_chunk)
            t_expanded = t_true.expand_as(candidate_heads_chunk)

            # Score the chunk of candidate triples (candidate_head, r_true, t_true)
            scores_chunk = complex_model(candidate_heads_chunk, r_expanded, t_expanded)
            all_head_scores.append(scores_chunk)

        # Concatenate scores for all candidates and find rank of the true head
        scores_all_heads = torch.cat(all_head_scores)
        rank_h = (scores_all_heads > scores_all_heads[h_true]).sum().item() + 1
        ranks.append(rank_h)
        for k in k_values:
            if rank_h <= k:
                hits_at_k[k] += 1

    # Aggregate results
    mean_rank = sum(ranks) / len(ranks) if len(ranks) > 0 else 0.0
    total_evaluated_predictions = num_eval_triples * 2 # Each original sample leads to head and tail prediction

    final_hits = {}
    for k in k_values:
        final_hits[f"Hits@{k}"] = hits_at_k[k] / total_evaluated_predictions

    results = {"Mean Rank": mean_rank}
    results.update(final_hits)

    return results

print("Defined evaluate_link_prediction_complex function for evaluation against all entities.")

Defined evaluate_link_prediction_complex function for evaluation against all entities.


**Reasoning**:
Now that the `evaluate_link_prediction_complex` function has been redefined to correctly handle link prediction evaluation for ComplEx against all possible entities, I will call this function to compute the Hits@k and Mean Rank metrics on the test set and print the summary results.

